In [ ]:
import requests
import json
from time import sleep


# TEST CONVEYOR SCRIPT

## Pre-requisites
ShopCX/Vizix

HUB connected & configured

RFID reader connected to the HUB & configured

ASN id

Tags 

## Principle

Request the HUB to compare the tags in the fields of the antennas & and the expected items in the reference list (ASN)

## Error Codes

0   Device is ready

100 Device Unknown 

101 Device KO 

102	Error, device already running

200	ASN Unknown

201	NOK: Expected and read items does not match

400	extended filter unknown

401	epcisresult filter unknown (only if device OK)

201	ASN already under processing

500	Unexpected error during the process

501	No EPCIS mapping for device

502	Failed to reach the hub

503	Connection with device closed early. Missing tags probable


In [ ]:
hubIp = ''
port =  '8081'

headers = {'accept': 'application/json'}
headerWithJson = {'Content-Type':'application/json','accept': 'application/json'}
requestTimeOut = 10

In [ ]:
# check devices
url = 'http://' + hubIp + ':' + '/supply-process/rest/devices'
print ("GET : "+url)
response = requests.get(url,headers=headers,verify=True, timeout=requestTimeOut)
if response.status_code == 200:
    print(json.dumps(response.json(),indent=4))
elif response.status_code == 404:
    print(response.content.decode("utf-8"))
else:
    print(response.content.decode("utf-8"))

# Default Process example

In [ ]:
# Process properties
deviceId = 'myDevice'
processTime = 5*1000
asnId = 'myAsnId'

In [ ]:
# start process
status = false
payload ={
}
url = 'http://' + hubIp + ':' + '/supply-process/rest/devices/'+deviceId+'/start?asnId='+asnId
print ("POST : "+url)
response = requests.post(url, headers=headerWithJson,json=payload, verify=True, timeout=requestTimeOut)
if response.status_code == 200:
    jsonResponse = response.json()
    print(json.dumps(response.json(),indent=4))
    statusCode = jsonResponse.get('code')
    if (!statusCode):
        print('KO Not Status Code !' )

    else if (statusCode==0):
        print('OK :'+jsonResponse.get('messsage'))
        status = True
    else:
        print('KO because :'+jsonResponse.get('messsage'))
    
elif response.status_code == 404:
    print(response.content.decode("utf-8"))
else:
    print(response.content.decode("utf-8"))

if (status):    
    # simulate the conveyor actions/timeout
    sleep(processTime)
    
    # stop process   
    url = 'http://' + hubIp + ':' + '/supply-process/rest/devices/'+deviceId+'/stop?epcisresult=all'
    print ("POST : "+url)
    response = requests.post(url, headers=headers, verify=True, timeout=requestTimeOut)
    if response.status_code == 200:
        jsonResponse = response.json()
        print(json.dumps(response.json(),indent=4))
        statusCode = jsonResponse.get('code')
        if (!statusCode):
            print('KO Not Status Code !' )
        else if (statusCode==0):
            print('OK :'+jsonResponse.get('messsage'))
            print(json.dumps(response.json(),indent=4))
        else if (statusCode==201):
            print('NOK :'+jsonResponse.get('messsage'))
            print(json.dumps(response.json(),indent=4))
        else:
            print('KO because :'+jsonResponse.get('messsage'))
    
    elif response.status_code == 404:
        print(response.content.decode("utf-8"))
    else:
        print(response.content.decode("utf-8"))